# Lamini Classifier Example
First let's install the Lamini Python SDK.

In [1]:
pip install lamini --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.7/697.7 kB 3.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


Now let's import the Lamini Python SDK and set our API key. You can get your API key from app.lamini.ai - every user gets $300 in free credits to get started.

In [2]:
import lamini

lamini.api_key = "<api-key> goes here"

Now we're all set up. The first thing we do is define a classifier - Lamini uses the concept of a "project" to contain one or many classifier versions for a given task.

When you create a project, you define several key elements that can't be changed later:

1. A project name
2. A set of categories or classes that you need to categorize
3. Descriptions of each category

Projects also (eventually) contain one or many versions of:

1. Example inputs and the expected output - we call this a dataset
2. Models trained on the dataset(s) you've provided

Below we're creating a project named "ClassifierExample" with a random suffix (just in case you're creating a few!).


In [3]:
from lamini.classify.lamini_classifier import LaminiClassifier
import random

cls = LaminiClassifier(
    f"ClassifierExample{random.randint(1000,9999)}"
)

Once the project is created, we define the classes. The more detailed the description, the higher your accuracy will be.

In [4]:

classes = {
    "cat": """The cat (Felis catus), also referred to as domestic cat or house cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. Valued by humans for companionship and its ability to kill vermin, the cat's retractable claws are adapted to killing small prey like mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, purring, trilling, hissing, growling, and grunting–as well as body language. It can hear sounds too faint or too high in frequency for human ears, such as those made by small mammals. It secretes and perceives pheromones.
            Domain:	Eukaryota
            Kingdom:	Animalia
            Phylum:	Chordata
            Class:	Mammalia
            Order:	Carnivora
            Suborder:	Feliformia
            Family:	Felidae
            Subfamily:	Felinae
            Genus:	Felis
            Species:	F. catus[1]""",
    "dog": """The dog is a domesticated descendant of the wolf. Also called the domestic dog, it was selectively bred from an extinct population of wolves during the Late Pleistocene by hunter-gatherers. The dog was the first species to be domesticated by humans, over 14,000 years ago and before the development of agriculture. 
            Domain:	Eukaryota
            Kingdom:	Animalia
            Phylum:	Chordata
            Class:	Mammalia
            Order:	Carnivora
            Family:	Canidae
            Genus:	Canis""",
}


Adding example inputs is optional, but will also help with accuracy. You can always do this later - we'll show you how later in this notebook.

In [5]:

examples = {
    "cat": [
        "Tend to be independent and aloof.",
        "Territorial and defensive .",
        "Self-grooming animals, using their tongues to keep their coats clean and healthy.",
        "Use body language and vocalizations, such as meowing and purring, to communicate."
    ],
    "dog": [
        "Social, pack-oriented, and tend to be more loyal to their human family.",
        "Need regular grooming from their owners, including brushing and bathing.",
        "Bark and growl to convey their messages.",
        "Responsive to human commands and can be trained to perform a wide range of tasks."
    ],
}


Now we initialize the project. This can take about a minute per class, so we'll put in a simple timer to keep us updated on status.

In [6]:
resp = cls.initialize(classes, examples) 

import time

while True:
    print("Waiting for classifier to initialize")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        first_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp["status"])
        raise Exception("failed training")


Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Waiting for classifier to initialize
Model ID: 77bccbac-390a-4a9b-98c6-602f4db8502d


Cool, we have our first model version! Let's try it out with a quick test.

In [9]:
import json

print(json.dumps(cls.classify("grrrr"), indent=2))


{
  "classification": [
    [
      {
        "class_id": 0,
        "class_name": "cat",
        "prob": 0.5164344412265378
      },
      {
        "class_id": 1,
        "class_name": "dog",
        "prob": 0.48356555877346213
      }
    ]
  ]
}


Now we can see how useful the classifier output is. We get a list of all the categories we defined in our project, plus a confidence score for each.

We can go even further to easily quantify the accuracy of our classifier. Let's run an evaluation!

What an evaluation means for a classifier: when you provide a set of inputs and the expected output, we can test the accuracy of the model on those inputs, and give you back both overall metrics as well as per-input assessment. 

In [10]:
from lamini.one_evaler.one_evaler import LaminiOneEvaler

eval = LaminiOneEvaler(
    test_model_id=first_model_id,
    eval_data_id=f"first_eval{random.randint(1000,9999)}",
    eval_data=[{"input": "woof", "target": "dog"}, {"input": "meow", "target": "cat"}],
    test_eval_type="classifier",
)

print(json.dumps(eval.run(), indent=2))


{
  "eval_job_id": "640937869",
  "eval_data_id": "first_eval1951",
  "metrics": {
    "tuned_accuracy": 1.0,
    "tuned_precision": 1.0,
    "tuned_recall": 1.0,
    "tuned_f1": 1.0
  },
  "status": "COMPLETED",
  "predictions": [
    {
      "input": "woof",
      "target": "dog",
      "test_output": "dog",
      "base_output": null
    },
    {
      "input": "meow",
      "target": "cat",
      "test_output": "cat",
      "base_output": null
    }
  ]
}


That first run was ok, but we can do better. Let's add some more examples and retrain to improve accuracy. You control when to add data and when to train.

In [11]:

resp = cls.add(
    "additional_data",
    {
        "cat": [
            "Cats spend up to sixteen hours a day sleeping, making them some of nature's most dedicated nappers.",
            "Felines possess an extraordinary sense of balance thanks to their flexible backbone and righting reflex.",
            "A cat's sandpaper-like tongue is covered in tiny hooks called papillae that help them groom themselves effectively.",
            "Female cats tend to be right-pawed while male cats are more often left-pawed, according to scientific studies.",
            "Ancient Egyptians showed their devotion to cats by mummifying them alongside their human companions.",
        ],
        "dog": [
            "Dogs have evolved alongside humans for over 15,000 years, developing an uncanny ability to read our facial expressions and emotions.",
            "The average dog can understand around 165 different words or signals, though some exceptional dogs can learn many more.",
            "A dog's sense of smell is roughly 40 times greater than a human's, allowing them to detect diseases and track scents that are days old.",
            "Unlike humans who have three cones in their eyes, dogs only have two, making them partially colorblind but excellent at detecting movement.",
            "The Basenji breed is known as the 'barkless dog' because it produces an unusual yodel-like sound instead of a typical bark.",
        ],
    },
)

resp = cls.train()

while True:
    print("Waiting for classifier to train")
    time.sleep(5)
    resp = cls.train_status()
    if resp["status"] == "completed":
        print("Model ID: " + resp["model_id"])
        second_model_id = resp["model_id"]
        break
    if resp["status"] == "failed":
        print(resp["status"])
        raise Exception("failed training")


Waiting for classifier to train
Waiting for classifier to train
Model ID: 2a1800dd-e9f4-4cc9-a0fc-8713e041f2f5


Great, now we have a second model version in our project! Let's run an eval and compare it to the first version.

In [12]:
print("Running comparison eval between model versions " + first_model_id + " and " + second_model_id)

eval_2 = LaminiOneEvaler(
    test_model_id=first_model_id,
    eval_data_id=f"second_eval{random.randint(1000,9999)}",
    eval_data=[{"input": "woof", "target": "dog"}, {"input": "meow", "target": "cat"}],
    test_eval_type="classifier",
    base_model_id=second_model_id,
    sbs=True,
    fuzzy=True,
)

print(json.dumps(eval_2.run(), indent=2))

Running comparison eval between model versions 77bccbac-390a-4a9b-98c6-602f4db8502d and 2a1800dd-e9f4-4cc9-a0fc-8713e041f2f5
{
  "eval_job_id": "1761500469",
  "eval_data_id": "second_eval4884",
  "metrics": {
    "base_accuracy": 1.0,
    "base_precision": 1.0,
    "base_recall": 1.0,
    "base_f1": 1.0,
    "base_fuzzy_accuracy": 1.0,
    "base_fuzzy_precision": 1.0,
    "base_fuzzy_recall": 1.0,
    "base_fuzzy_f1": 1.0,
    "tuned_accuracy": 1.0,
    "tuned_precision": 1.0,
    "tuned_recall": 1.0,
    "tuned_f1": 1.0,
    "tuned_fuzzy_accuracy": 1.0,
    "tuned_fuzzy_precision": 1.0,
    "tuned_fuzzy_recall": 1.0,
    "tuned_fuzzy_f1": 1.0,
    "tuned_win_loss_ratio": 0.0,
    "base_win_loss_ratio": 0.0
  },
  "status": "COMPLETED",
  "predictions": [
    {
      "input": "woof",
      "target": "dog",
      "test_output": "dog",
      "base_output": "dog"
    },
    {
      "input": "meow",
      "target": "cat",
      "test_output": "cat",
      "base_output": "cat"
    }
  ]
}


The eval output makes it easy to compare model versions overall, and to see exactly where the differences are, so you know exactly where to focus to improve your workflow.

Happy classifying!